# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [17]:
!pip install -r "/content/requirements.txt"
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.4/644.4 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: openai
    Found existing installation: openai 1.72.0
    Uninstalling openai-1.72.0:
      Successfully uninstalled openai-1.72.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [56]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [ ]:
#!pip install pandas

In [7]:
import pandas as pd
df = pd.read_csv('/content/Data.csv')

In [ ]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [8]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00


In [18]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [23]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.3)


In [24]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Write a query that describes the product: {product}"
)


In [25]:

chain = LLMChain(llm=llm, prompt=prompt)

<ipython-input-25-726db833e4be>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [26]:
product = "smartwatch with health tracking features"
chain.run(product)


<ipython-input-26-10732ad4953f>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


"SELECT * \nFROM products \nWHERE description LIKE '%smartwatch%' \nAND description LIKE '%health tracking%';"

## SimpleSequentialChain

In [27]:
from langchain.chains import SimpleSequentialChain

In [28]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Write a detailed product description for: {product}"
)


# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [29]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the following product description, write a catchy marketing tagline:\n\n{description}"
)

# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [30]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [31]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Introducing our revolutionary smartwatch with advanced health tracking features designed to help you stay on top of your wellness goals. 

This sleek and stylish smartwatch boasts a range of cutting-edge health tracking capabilities that will empower you to take control of your health like never before. Equipped with a built-in heart rate monitor, step counter, and sleep tracking feature, this smartwatch provides real-time insights into your physical activity levels and overall well-being.

Whether you're an avid athlete or simply looking to improve your fitness, this smartwatch has you covered. With customizable fitness goals and workout tracking, you can easily track your progress and stay motivated to reach new milestones. The smartwatch even offers guided breathing exercises and relaxation reminders to help you manage stress and improve your mental health.

But that's not all - this smartwatch also includes a comprehensive health moni

'"Track your health, reach your goals, and stay connected with our revolutionary smartwatch!"'

**Repeat the above twice for different products**

## SequentialChain

In [32]:
from langchain.chains import SequentialChain

In [33]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
    "Translate the following product review into English:\n\n{review}"
)


chain_one = LLMChain(
    llm=llm,
    prompt=first_prompt,
    output_key="translated_review"
)



In [34]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following product review in one or two sentences:\n\n{translated_review}"
)


chain_two = LLMChain(
    llm=llm,
    prompt=second_prompt,
    output_key="summary"
)



In [35]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate the following product review to {language}:\n\n{review}"
)

# chain 3: input= Review and output= language
chain_three = LLMChain(
    llm=llm,
    prompt=third_prompt,
    output_key="translation"
)

inputs = {
    "review": "Ce produit est fantastique, je le recommande vivement.",
    "language": "English"
}

result = chain_three.run(inputs)
print(result)



This product is fantastic, I highly recommend it.


In [36]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
    "Based on the following translated review:\n\n{translated_review}\n\n"
    "And this summary:\n\n{summary}\n\n"
    "Write a friendly follow-up message to the customer."
)

chain_four = LLMChain(
    llm=llm,
    prompt=fourth_prompt,
    output_key="followup_message"
)



In [40]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_four],
    input_variables=["review"],
    output_variables=["translated_review", "summary", "followup_message"],
    verbose=True
)



In [41]:
review = df.Review[5]
overall_chain(review)

<ipython-input-41-b19509cc8fc9>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'translated_review': "I find the taste mediocre. The foam doesn't hold up, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer is disappointed in the taste and foam quality of the product, suspecting that it may be an old batch or counterfeit compared to their usual store-bought version.',
 'followup_message': "Hi there,\n\nWe're sorry to hear that you were disappointed with your recent purchase of our product. Your feedback is important to us and we want to ensure that you receive the quality you expect from our brand. We would like to further investigate this issue to see if there was a batch or quality control issue. Please feel free to reach out to our customer service team so we can address your concerns and make things right. Th

**Repeat the above twice for different products or reviews**

## Router Chain

In [42]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [43]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [44]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [45]:
llm = ChatOpenAI(temperature=0)

In [46]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [47]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [48]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [49]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [50]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

<ipython-input-50-ae9cc652db54>:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [51]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [52]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [53]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell division, and repair. Having DNA in every cell allows for the coordination of these processes and ensures that the organism functions properly as a whole. \n\nIn summary, every cell in our body contains DNA because it is essential for the proper functioning and development of all living organisms.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [54]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.7)

# Step 1: Translate Tweet
prompt_translate = ChatPromptTemplate.from_template(
    "Translate this tweet to English:\n{input}"
)
chain_translate = LLMChain(llm=llm, prompt=prompt_translate, output_key="english_tweet")

# Step 2: Detect Topic
prompt_topic = ChatPromptTemplate.from_template(
    "What is the main topic of this tweet? (e.g., politics, sports, tech, entertainment, etc):\n{english_tweet}"
)
chain_topic = LLMChain(llm=llm, prompt=prompt_topic, output_key="topic")

# Step 3: Generate a trendy hashtag
prompt_hashtag = ChatPromptTemplate.from_template(
    "Create a short and trendy hashtag for a tweet about {topic}."
)
chain_hashtag = LLMChain(llm=llm, prompt=prompt_hashtag, output_key="hashtag")

# Combine into a SequentialChain
tweet_chain = SequentialChain(
    chains=[chain_translate, chain_topic, chain_hashtag],
    input_variables=["input"],
    output_variables=["hashtag"],
    verbose=True
)

# Example foreign-language tweet (in Spanish)
tweet = "La inteligencia artificial está cambiando el mundo más rápido de lo que pensábamos."

result = tweet_chain.run({"input": tweet})
print("\nFinal Hashtag:\n", result)




> Entering new SequentialChain chain...

> Finished chain.

Final Hashtag:
 #TechTrends2021


In [55]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.85)

# Step 1: Turn dream into image prompt
prompt_to_image = ChatPromptTemplate.from_template(
    "Convert this dream journal entry into a detailed image generation prompt:\n{input}"
)
chain_img_prompt = LLMChain(llm=llm, prompt=prompt_to_image, output_key="image_prompt")

# Step 2: Generate an Instagram caption for the dream-art
prompt_caption = ChatPromptTemplate.from_template(
    "Write a poetic Instagram caption for an artwork with this description:\n{image_prompt}"
)
chain_caption = LLMChain(llm=llm, prompt=prompt_caption, output_key="caption")

# Create the chain
dream_to_instagram_chain = SequentialChain(
    chains=[chain_img_prompt, chain_caption],
    input_variables=["input"],
    output_variables=["caption"],
    verbose=True
)

# Example dream
dream_text = """
I was walking on clouds made of cotton candy,
giant koi fish swam through the sky,
and a whispering moon followed me wherever I went.
"""

result = dream_to_instagram_chain.run({"input": dream_text})
print("\n🌙 Final Instagram Caption:\n", result)




> Entering new SequentialChain chain...

> Finished chain.

🌙 Final Instagram Caption:
 "Lost in a dream world of cotton candy clouds and celestial koi fish, where the moon whispers secrets of magic and whimsy. 🌙✨ #Dreamscape #SurrealArt"
